In [ ]:
# 로컬 환경에서 동작하도록 수정된 챗봇 코드
import json
import os
from pathlib import Path

# JSON 파일 탐색 함수: 여러 후보 경로를 시도합니다.
def find_json_path():
    candidates = [
        Path("YachtList01.json"),
        Path.cwd() / "YachtList01.json",
        Path.cwd() / "front" / "YachtList01.json",
        Path(r"c:\\Users\\user\\Documents\\요트앱\\front\\YachtList01.json"),
    ]

    for p in candidates:
        try:
            if p.exists():
                return p.resolve()
        except Exception:
            continue
    return None

json_path = find_json_path()
if json_path is None:
    raise FileNotFoundError(
        "YachtList01.json 파일을 찾을 수 없습니다. 노트북(.ipynb)과 동일한 폴더에 'YachtList01.json'을 놓거나 경로를 확인하세요."
    )

with open(json_path, 'r', encoding='utf-8') as f:
    yacht_data = json.load(f)


def get_yacht_dimensions_by_question(question):
    # 크기 관련 키워드 확인 (모두 소문자로 비교)
    keywords = ['크기', '길이', '전장', 'lwl', 'loa', 'beam', '폭', 'draft', '흘수']
    question_lower = question.lower()
    if not any(k in question_lower for k in keywords):
        return None

    # 모델명 매칭: 질문에 포함된 모델명과 부분 일치
    for yacht in yacht_data.get('yachts', []):
        model_name = yacht.get('model_name', '').lower()
        if not model_name:
            continue
        if model_name in question_lower or any(part in question_lower for part in model_name.split()):
            dim = yacht.get('dimensions', {})
            reply = (
                f"'{yacht.get('model_name', '?')}'의 크기 정보는 아래와 같습니다:\n"
                f"- LOA(전장): {dim.get('LOA_m', '?')} m ({dim.get('LOA_ft', '?')} ft)\n"
                f"- LWL(수선간장): {dim.get('LWL_m', '?')} m ({dim.get('LWL_ft', '?')} ft)\n"
                f"- Beam(폭): {dim.get('Beam_m', '?')} m ({dim.get('Beam_ft', '?')} ft)\n"
                f"- Draft(흘수): {dim.get('Draft_m', '?')} m ({dim.get('Draft_ft', '?')} ft)"
            )
            return reply
    return "해당 요트 정보를 찾을 수 없습니다. 모델명을 정확히 입력해주세요."


def chatbot():
    print("요트 상태 질문 챗봇입니다. '종료'를 입력하면 종료됩니다.")
    while True:
        user_input = input("질문을 입력하세요: ").strip()
        if user_input == '종료':
            print("챗봇을 종료합니다.")
            break

        answer = get_yacht_dimensions_by_question(user_input)
        print("챗봇:", answer or "해당 질문에 맞는 응답을 준비하지 못했습니다.")

# 주의: 노트북에서 자동 실행을 피하기 위해 chatbot()는 자동 호출하지 않습니다.
# 노트북에서 직접 시험해보려면 아래 테스트 셀을 실행하세요.


In [ ]:
# 테스트 셀: JSON 로드 상태 확인 및 기본 동작 확인
print(f"사용된 JSON 경로: {json_path}")
print(f"총 요트 개수: {len(yacht_data.get('yachts', []))}")

# 상위 5개 모델명 출력
for i, y in enumerate(yacht_data.get('yachts', [])[:5], start=1):
    print(f"{i}. {y.get('model_name', '(이름없음)')}")

# 예시 질의 테스트 (데이터가 있으면 첫 요트 모델명을 사용)
if yacht_data.get('yachts'):
    first_model = yacht_data['yachts'][0].get('model_name', '')
    if first_model:
        sample_question = f"{first_model}의 길이는 얼마인가요?"
        print('\n샘플 질문:', sample_question)
        print('응답:')
        print(get_yacht_dimensions_by_question(sample_question))
